##### Objective

Notebook covers the implementation and notes linked with the Queue and Stack learnt from https://realpython.com/queue-in-python/

Most of the implementation might be implemented in multiple cells, and in some cases the code would get written as a file to the drive but the code will be here for review. 

### Unlike the other implementations, this notebook will contain the usage of existing python module like Collections, Itertools to implement/ understand the logic

That means, there will be no seperate Class Type for Node that are pushed into Queue or Stack. 

In [2]:
### Implementing custom logger for practice
import logging

quelog = logging.getLogger('quelog')
quelog.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
handler.setLevel(logging.INFO)
logf = logging.Formatter(fmt='%(message)s | %(asctime)s | %(name)s', datefmt='%d-%b | %H-%M')
handler.setFormatter(logf)
quelog.addHandler(handler)

# testing the quelog
quelog.info("Pushing the test message out.")

Pushing the test message out. | 21-Nov | 13-17 | quelog


In [2]:
from collections import deque

class Queue:
    def __init__(self):
        # observe the private element usage
        self._elements = deque()

    def enqueue(self, elem):
        # only used for appending the elements, no return
        self._elements.append(elem)

    def dequeue(self):
        # only returns the element from the left of the queue to 
        # simulate the fifo config
        return self._elements.popleft()

In [6]:
# checking the above implementation

fifo = Queue()

fifo.enqueue('1st')
fifo.enqueue('2st')
fifo.enqueue('3rd')

In [7]:
quelog.warning(fifo.dequeue())  # 1st
quelog.warning(fifo.dequeue())  # 2nd
quelog.warning(fifo.dequeue())  # 3rd
quelog.warning(fifo.dequeue())  # must throw indexerror 

1st | 21-Nov | 06-21 | quelog
2st | 21-Nov | 06-21 | quelog
3rd | 21-Nov | 06-21 | quelog


IndexError: pop from an empty deque

In [12]:
# implementing iter and len methods, with optional initial elements

from collections import deque

class Queue01:
    def __init__(self, *elems) -> None:
        self._elements = deque(elems)

    def __len__(self):
        return len(self._elements)

    def __iter__(self):
        # note, after the __len__ elem is implemented, the len(self) will return length
        while len(self) > 0:
            yield self.dequeue() # note this dequeue method is implemented below for self

    def enqueue(self, elem):
        self._elements.append(elem)

    def dequeue(self):
        return self._elements.popleft()

In [18]:
newFifo = Queue01('5th', '7th', '15th')

In [19]:
for elem in newFifo:
    quelog.info(elem)

5th | 21-Nov | 06-34 | quelog
INFO:quelog:5th
7th | 21-Nov | 06-34 | quelog
INFO:quelog:7th
15th | 21-Nov | 06-34 | quelog
INFO:quelog:15th


In [20]:
# Building the stack is done through inheritance 

class Stack(Queue01):
    def dequeue(self):
        # elements are instantiated using the super class _elements
        return self._elements.pop()

In [21]:
lifo = Stack("6th", '8th', '10th')

for ele in lifo:
    quelog.info(ele)

10th | 21-Nov | 06-39 | quelog
INFO:quelog:10th
8th | 21-Nov | 06-39 | quelog
INFO:quelog:8th
6th | 21-Nov | 06-39 | quelog
INFO:quelog:6th


### Above inheritance is done only for the code reuse. Stack is not a subtype of Queue.  In the real world, you should probably make both classes inherit from an abstract base class because they share the same interface.

Implementing Priority Queue:

In a priority queue, it’s an element’s priority and the insertion order that together determine the ultimate position within the queue.
So the above Queue or Queue01 cannot be reused

An unordered list of elements and their priorities, which you search through every time before dequeuing the element with the highest priority

An ordered list of elements and their priorities, which you sort every time you enqueue a new element

An ordered list of elements and their priorities, which you keep sorted by finding the right spot for a new element using binary search

A binary tree that maintains the heap invariant after the enqueue and dequeue operations

Based on the time complexity involved for sorting with different data structure, the heap datastructure is used

Python has the heapq module, which conveniently provides a few functions that can turn a regular list into a heap and manipulate it efficiently. The two functions that’ll help you build a priority queue are:

heapq.heappush()

heapq.heappop()

- Heap isn’t so much about sorting elements but rather keeping them in a certain relationship to allow for "quick lookup". 

- first element on a heap always has the smallest (min-heap) or the highest (max-heap) value. 

In [25]:
from heapq import heappush

fruits = []

heappush(fruits, 'orange')
heappush(fruits, 'apple')
heappush(fruits, 'banana')

fruits


['apple', 'orange', 'banana']

In [26]:
from heapq import heappop

quelog.info(heappop(fruits))

apple | 21-Nov | 07-27 | quelog
INFO:quelog:apple


In [28]:
quelog.info(fruits)  # first element continues to be smaller

# character with a lower Unicode code point is considered smaller in Python

['banana', 'orange'] | 21-Nov | 07-28 | quelog
INFO:quelog:['banana', 'orange']


In [30]:
person_1 = ('Jane', 'doe', 57)
person_2 = ('Jane', 'smith', 38)
person_3 = ('Noah', 'Smith', 27)

person_1 < person_2  # How python decides which tuple to go first? 

# Once the order is decided (last name in this case), rest of the elements don't matter

# You can enforce a prioritized order on the heap by storing tuples whose first element is a priority. 

True

In [31]:
# implementing priority queue with deque and heapq

class PriorityQueue:
    def __init__(self):
        self._elements = []

    def enqueue_with_priority(self, priority, value):
        heappush(self._elements, (priority, value))

    def dequeue(self):
        return heappop(self._elements)

In [35]:
# priority items

CRITICAL = 3
IMPORTANT = 2
NEUTRAL = 1

# priority values can be ordered in the way implementation wants. If the order changes a new class can be implemented

In [33]:
messages = PriorityQueue()

messages.enqueue_with_priority(IMPORTANT, 'windshield wipers')
messages.enqueue_with_priority(NEUTRAL, 'Radio station tuning')
messages.enqueue_with_priority(CRITICAL, 'Break Pedal depressed')
messages.enqueue_with_priority(CRITICAL, 'Airbag infate')

In [36]:
messages.dequeue()  # observe the neutral element is coming out first... which can be undesirable when priority values are higher

(2, 'windshield wipers')

In [38]:
# Implementing another variation of the class 

class PriorityQueue:
    def __init__(self) -> None:
        self._elements = []

    def enqueue_with_priority(self, priority, value):
        # making the priority negative changes how it is placed in the heapq
        heappush(self._elements, (-priority, value))

    def dequeue(self):
        return heappop(self._elements)[1] # okay, that tripped me. We are returning a tuple remember

In [39]:
messages = PriorityQueue()

messages.enqueue_with_priority(IMPORTANT, 'windshield wipers')
messages.enqueue_with_priority(NEUTRAL, 'Radio station tuning')
messages.enqueue_with_priority(CRITICAL, 'Break Pedal depressed')
messages.enqueue_with_priority(CRITICAL, 'Airbag infate')

In [40]:
messages.dequeue()

'Airbag infate'

In [41]:
messages.dequeue()

'Break Pedal depressed'

Even though the priority queue above keeps the order correctly, but the sorting is unstable. When messages have the same priority the queue should sort them by their insertion order.
This inconsistency happens due to the lexicographic order of comparison between elements in python. So the word Hazard comes before Windshield, which creates the challenge

In [1]:
from dataclasses import dataclass

@dataclass
class Message:
    event: str

wipers = Message("Windshield On")
right = Message("Right light On")

wipers < right

TypeError: '<' not supported between instances of 'Message' and 'Message'

Message objects might be more convenient to work with than plain strings, but unlike strings, they aren’t comparable unless you tell Python how to perform the comparison.

To avoid the comparison of strings, the alternate is to use timestamp. A monotonically increasing counter will do the trick. In other words, you want to count the number of enqueue operations performed without considering the potential dequeue operations that might be taking plac

In [2]:
from collections import deque
from heapq import heappop, heappush
from itertools import count

class PriorityQueue:
    def __init__(self) -> None:
        self._elements = []
        self._counter = count()

    def enqueue_with_priority(self, priority, value):
        elem = (-priority, next(self._counter), value)
        heappush(self._elements, elem)

    def dequeue(self):
        return heappop(self._elements)[-1]

To reuse the code in Python, you find the least common denominator and then extract that code into a MixinClass

In [4]:
class IterableMixin:
    def __len__(self):
        return len(self._elements)

    def __iter__(self):
        # note, after the __len__ elem is implemented, the len(self) will return length
        while len(self) > 0:
            yield self.dequeue() # note this dequeue method is implemented below for self

# To emphasize this difference, some people call it the inclusion of a mixin class rather than pure inheritance.


In [5]:
from collections import deque
from heapq import heappop, heappush
from itertools import count
# Mixins are great for encapsulating behavior rather than state, much like default methods in Java interfaces.
class PriorityQueue(IterableMixin):
    def __init__(self) -> None:
        self._elements = []
        self._counter = count()

    def enqueue_with_priority(self, priority, value):
        elem = (-priority, next(self._counter), value)
        heappush(self._elements, elem)

    def dequeue(self):
        return heappop(self._elements)[-1]

python -m pip install networkx pygraphviz

In [3]:
import networkx as nx

quelog.info(nx.nx_agraph.read_dot('materials-queue/src/roadmap.dot'))

ImportError: read_dot() requires pygraphviz http://pygraphviz.github.io/